# End-to-End Optimisation

The notebook you have been provided contains code for a convolutional neural network and an image pre-processing and augmentation pipeline. The network and data pipeline are poorly optimised. Your task for this assignment is to analyse the performance of the network, in terms of training speed and the quality of the results, and then make improvements to the network based on your findings.

You will be training the model on the Oxford-IIIT pet dataset. The pipeline has a preprocessing function that performs some data augmentation. Consider how the pipeline can be made more efficient, use the Tensorboard Profiler tool to help—note you only need to profile a couple of batches, not the full training process.

The CNN uses a VGGNet architechture, built up with blocks of convolutional layers and maxpooling layers. Consider the settings of the existing layers, and whether additional layers can be added to improve the model accuracy, speed up learning, and tackle issues like over fitting. Research decisions around activation functions, optimiser settings.

Train the model as is at least once, and analyse the results—to speed up the notebook for future runs, you can save the weights and reload them to perform analysis. If you do save trhe weights, be sure to download them or store them in your google drive for easy access later on. The initial model could take up to an hour to run, so plan your time ahead.

You should create plots of the loss and metric curves, and to demonstrate the predictive capabilities of the model—think about what types of images the model is getting correct and what it is getting wrong.

You might want to add in callbacks to control modify training, but this isn't strictly necessary.

(I can't stress enough that this model and pipeline are *bad* and that is deliberate. It should just about learn *something* with no changes, but it won't do a good job. You can change anything in the code below, except for the block that loads the dataset—no training the model on a different set of images. You should be aiming to improve three key aspects in order to achieve full marks—training speed, accuracy of the training dataset, and accuracy of the validation/testing dataset.)

Your task for this assignment is to analyse the performance of the network, in terms of training speed and the quality of the results, and then make improvements to the network based on your findings.

1.	Train the network and plot the resulting loss functions and metrics. Create plots that demonstrate the predictive results of the network. Comment on these results in your report. (10 marks)
2.	Address the performance issues in the image processing pipeline, comment on the choices you make in your report. (10 marks)
3.	Adjust the design of the CNN to achieve improved predictive results, comment on the choices you make in your report. (10 marks)
4.	Change the settings used in training the network to improve both the training speed and predictive results. (5 marks)
5.	Train the new network, and repeat the analysis performed in the first step. Comment in your report how the changes you made have improved the speed and performance of the network. (10 marks)
Along with your completed Notebook, you should produce a 2-page report as detailed above. (5 marks for writing quality)


# Exercise 1 (10 marks)

Train the network and plot the resulting loss functions and metrics. Create plots that demonstrate the predictive results of the network. Comment on these results in your report. If you are worried about space in your report, you can number your plots inside the notebook and refer to those numbers in the text.

In [ ]:
# Install the tensorflow addons package,
# which has a nice image rotation function
!pip install tensorflow-addons

In [ ]:
# Import modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

import numpy as np

In [ ]:
# Define some Important Variables
img_height = 128
img_width = 128
IMG_SIZE = (img_height,img_width)
BATCH_SIZE = 32
EPOCHS = 20

In [ ]:
# Image Preprocessing functions. You should aim to optimise these in Exercise 2.

def img_process_train(features):
    """
    A preprocessing fuction for the training dataset. This function accepts the
    oxford_iiit_pet dataset, extracts the images and species label, the performs
    random augmentations before resizing and rescaling the images.
    """
    image = features['image']
    label = features['species']

    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, 0.1)
    image = tf.image.random_contrast(image, 0.2, 0.5)
    image = tf.image.random_hue(image, 0.2)
    image = tf.image.random_jpeg_quality(image, 75, 95)
    image = tf.image.random_saturation(image, 5, 10)

    # Use tensorflow addons to randomly rotate images
    deg = np.random.uniform(-20,20)
    image = tfa.image.rotate(image, deg)

    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, 'float32')/255.

    return image, label

def img_process_test(features):
    """
    A preprocessing fuction for the test and validation datasets. This function
    accepts the oxford_iiit_pet dataset, extracts the images and species label,
    and resizes and rescales the images.
    """
    image = tf.image.resize(features['image'], IMG_SIZE)
    image = tf.cast(image, 'float32')/255.
    return image, features['species']


In [ ]:
# Load the dataset

### DONT CHANGE ME ###
train_ds, val_ds, test_ds = tfds.load(
"oxford_iiit_pet",
split=["train[:100%]", "test[:50%]", "test[50%:100%]"],
)

### DONT CHANGE ME ###

# Set up the datasets with the augmentations and resizing
train_ds = train_ds.map(img_process_train).batch(BATCH_SIZE)
val_ds = val_ds.map(img_process_test).batch(BATCH_SIZE)
test_ds = test_ds.map(img_process_test).batch(BATCH_SIZE)

In [ ]:
# Sequential Model Definition
def create_model_unoptimised():
    model = Sequential([
    layers.Input(shape=IMG_SIZE+(3,), name='Input'),
    layers.Conv2D(32, 4, padding='same', activation='relu',
                    name='Conv_1'),
    layers.Conv2D(32, 4, padding='same', activation='relu',
                    name='Conv_2'),
    layers.MaxPooling2D(name='Pool_1'),
    layers.Conv2D(64, 3, padding='same', activation='relu',
                    name='Conv_3'),
    layers.Conv2D(64, 3, padding='same', activation='relu',
                    name='Conv_4'),
    layers.MaxPooling2D(name='Pool_2'),
    layers.Flatten(name='Flatten'),
    layers.Dense(512, activation='relu', name='dense_1'),
    layers.Dense(1, name='Output')
    ], name='CNN')

    return model

In [ ]:
# Create a version of the model and print the summary
model = create_model_unoptimised()

model.summary()

In [ ]:
# Adam optimiser
opt = tf.keras.optimizers.Adam()
# Binary classification loss
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# Accuracy metric
metrics = ['accuracy']

# Compile model
model.compile(optimizer=opt,
              loss=loss_obj,
              metrics=metrics)


In [ ]:
# Train the Model
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=EPOCHS)


# Exercise 2 (10 marks)
Address the performance issues in the image processing pipeline, comment on the choices you make in your report.

You can copy the code above and make changes, or write the piepline from scratch. This could include creating new preprocessing functions.

Use this exercise to change the definitions of the training, validation and testing datasets to improve the speed at which the network trains, ***and*** predictive performance.

In [ ]:
# Load the dataset

### DONT CHANGE ME ###
train_ds, val_ds, test_ds = tfds.load(
"oxford_iiit_pet",
split=["train[:100%]", "test[:50%]", "test[50%:100%]"],
)

### DONT CHANGE ME ###

In [ ]:
### Edit the code below to improve the training speed and predictive performance
### of the network. Consider how to better implement the img_process_train and
### img_process test fuctions.

# Set up the datasets with the augmentations and resizing
train_ds = train_ds.map(img_process_train).batch(BATCH_SIZE)
val_ds = val_ds.map(img_process_test).batch(BATCH_SIZE)
test_ds = test_ds.map(img_process_test).batch(BATCH_SIZE)

# Exercise 3 (10 marks)
Adjust the design of the CNN to achieve improved predictive results, comment on the choices you make in your report.

You can add in new layers, change the settings of the existing layers. You could even use a different CNN architecture. Consider how the layer settings contribute to slowing down the network, such as with very large calculations with high numbers of traininable parameters.

Comment on any changes you make in your report, including details of any tests you performed. You might find it useful to create a new notebook specifically for testing different models, so that this notebook doesn't get too clogged up with outputs.

In [ ]:
### Change the model architecture to improve both the training speed and
### predictive performance

# Sequential Model Definition
def create_model_optimised():
    model = Sequential([
    layers.Input(shape=IMG_SIZE+(3,), name='Input'),
    layers.Conv2D(32, 4, padding='same', activation='relu',
                    name='Conv_1'),
    layers.Conv2D(32, 4, padding='same', activation='relu',
                    name='Conv_2'),
    layers.MaxPooling2D(name='Pool_1'),
    layers.Conv2D(64, 3, padding='same', activation='relu',
                    name='Conv_3'),
    layers.Conv2D(64, 3, padding='same', activation='relu',
                    name='Conv_4'),
    layers.MaxPooling2D(name='Pool_2'),
    layers.Flatten(name='Flatten'),
    layers.Dense(512, activation='relu', name='dense_1'),
    layers.Dense(1, name='Output')
    ], name='CNN')

    return model

# Exercise 4 (5 marks)

Change the settings used in training the network to improve both the training speed and predictive results.

Consider how you might compile the model with different settings, how many epochs the model needs to be trained for, how learning rates and batch sizes might affect training.

In [ ]:
### Experiment with the training settings to maximise the predictive performance
### of the network

# Adam optimiser
opt = tf.keras.optimizers.Adam()
# Binary classification loss
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# Accuracy metric
metrics = ['accuracy']

# Compile model
model.compile(optimizer=opt,
              loss=loss_obj,
              metrics=metrics)


# Exercise 5 (10 marks)

Train the new network, and repeat the analysis performed in the first step. Comment in your report how the changes you made have improved the speed and performance of the network.